This should all work out of the box, not much adjustment needed, just point filepath to wherever your images are and run it.

In [ ]:
import pandas as pd
import glob

In [ ]:
filepath = '/Volumes/maxdrive/AstroData/2023_1_23/Light/Blue/Reduced/' #Change to path of wherever your reduced images are

In [ ]:
flistfits = sorted(glob.glob(filepath+'Reduced_star_Light_*.fits')) #This probably won't need to be changed but check just in case you have different file name patterns

In [ ]:
import os
from astropy.io import fits
import glob
import astroalign as aa



refimage = flistfits[0]
with fits.open(refimage) as hdul:
    refimage_data = hdul[0].data.astype('<f4')
    refheader = hdul[0].header.copy()


for file in flistfits:
    with fits.open(file) as hdul:
        image_data = hdul[0].data.astype('<f4')
        aligned_image, footprint = aa.register(target=refimage_data, source=image_data)
        aligned_hdul = fits.PrimaryHDU(aligned_image, header=refheader)
        aligned_hdul.header.set('ORIGFILE', os.path.basename(file), 'Original filename')
        aligned_hdul.header.set('ORIGTIME', hdul[0].header.get('DATE-OBS'), 'Original observation time')
        fname = filepath+'Aligned/'+'aligned_'+(file.replace(filepath, ""))
        aligned_hdul.writeto((fname), overwrite=True)
        print (fname)
